# Fine-tune a german GPT-2 model with 31 years of iX articles

In this Notebook I'm going to fine-tune a German GPT-2 from the [Huggingface model hub](https://huggingface.co/models). As fine-tune I using the iX archive (1988 - 2019) from [here](https://shop.heise.de/ix-archiv-1988-2019-usb-stick-1) and convert the PDF and HTML data into pure UTF-8 text with this [converter](https://github.com/rawar/ix-archive-utils).
The text file contains 31.304 german artcile pages.
The idea is that I use the iX articles to fine-tune an existing, german GPT-2 model to let me write iX articles like a pro. The original idea of this notebook are published by [Philipp Schmid](https://www.philschmid.de/fine-tune-a-non-english-gpt-2-model-with-huggingface). The original notebook is stored [here](https://colab.research.google.com/github/philschmid/fine-tune-GPT-2/blob/master/Fine_tune_a_non_English_GPT_2_Model_with_Huggingface.ipynb#scrollTo=PGEiQ1mhOyNv) in Google Colaboration.

In [2]:
!/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip -q install --upgrade pip

## Install modules

In [3]:
!pip install -q transformers 

## Check GPU environment 
To check the GPU environment, I'm using the NVIDIA system management interface. 

In [4]:
!nvidia-smi

Sat Feb 13 15:24:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import time
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TextDataset,DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoModelWithLMHead
from transformers import pipeline

In [6]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 500)

In [7]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/german-gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [8]:
model = AutoModelWithLMHead.from_pretrained("dbmdz/german-gpt2")

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/transformers/models/auto/modeling_auto.py:970: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [9]:
configuration = model.config
print(configuration)

GPT2Config {
  "_name_or_path": "dbmdz/german-gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 52000,
  "embd_pdrop": 0.1,
  "eos_token_id": 52000,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.3.2",
  "use_cache": true,
  "vocab_size": 52000
}



In [10]:
data = pd.read_csv("./ix-archive.csv", sep='\n', header=None, encoding='utf-8')

In [11]:
data.head()

,0
0,"ix.0208.098-101 07.01.2008 15:30 Uhr Seite 100 REPORT Internet So ist beispielsweise die DSL-Flatrate eines T-DSL-Resale-Anbieters technisch nicht an den von ihm bereitgestellten DSL-Anschluss geknüpft. Gerade für kleinere DSL-Anbieter, die sich mit besonderen Features auf dem Markt positionieren, bringt dies entscheidende Vorteile. Sie können ihre Kunden unabhängig von deren Anschlussart versorgen. Wenig bekannt ist die Tatsache, dass der DSL-Anbieter nicht wissen muss, von welchem (Telefon..."
1,MARKT + TRENDS : Verschiedenes Verschiedenes Compaqs Alpha-Großrechner JavaScript-Bug im Navigator 4.x Elektronisches Papier marktreif NetCenter mit neuen Eigenschaften Symposium: Computer für alles und überall
2,"xx.1516.044-052.neu1.qxp 31.05.16 08:17 Seite 51 Im Normalfall geht man allerdings anders vor: Will man LibreOffice ausrollen und möchte hierzu ein bestimmtes Profil für die Nutzer haben, so erzeugt man zunächst eine saubere („clean“) Installation und lässt dann ein neues Profil (und somit auch eine initiale registrymodification.xcu) erzeugen. Nun korrigiert man in der Oberfläche alle Einstellungen, die später für alle Benutzer gelten sollen, beendet die Optionen und das Programm und hat nu..."
3,MARKT + TRENDS : Wirtschaft Wirtschaft DOMEA: Wahlkandidat für Behörden Aus USWeb und CKS wird Reinvent Helsinki Telephone steigt in Münster ein Siemens: SAP statt Peoplesoft Geht IBM-Chef zur Deutschen Bank? Kurz notiert
4,"ix.0810.096-100 05.07.2010 11:02 Uhr Seite 99 Routen häufiger in Benutzung sind als andere. Grundsätzlich können Datenpakete jeden möglichen Weg nehmen. Fallen Verbindungen aus oder sind überlastet, können die Router laufend Anpassungen vornehmen und Pakete umleiten. Dieses dynamische Routing kann Nachteile haben. Es ist allgemein bekannt, dass unverschlüsselte E-Mails wie eine Postkarte anzusehen sind, die jeder lesen kann, der Zugriff auf eines der Systeme hat, die die E-Mail transportiere..."


In [12]:
train, test = train_test_split(data,test_size=0.15)
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'
date_name = timestr = time.strftime("%Y%m%d-%H%M%S")
output_model_path = f"{date_name}_gpt2_ix"


In [13]:
train.to_csv(train_path, header=False, index=False)
test.to_csv(test_path, header=False, index=False)

In [14]:
print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 26345
Test dataset length: 4650


In [15]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_path,
        block_size=128
    )

    test_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=test_path,
        block_size=128
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

In [16]:
train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


## Training arguments
A list of all training arguments can you find [here](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). 

In [17]:
training_args = TrainingArguments(
    output_dir=output_model_path, 
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=20, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=50000, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True
)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [19]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  503616 KB |  503616 KB |  503616 KB |       0 B  |
|       from large pool |  490848 KB |  490848 KB |  490848 KB |       0 B  |
|       from small pool |   12768 KB |   12768 KB |   12768 KB |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |  503616 KB |  503616 KB |  503616 KB |       0 B  |
|       from large pool |  490848 KB |  490848 KB |  490848 KB |

In [ ]:
trainer.train()

Step,Training Loss
500,4.310900
1000,3.984100
1500,3.883900
2000,3.818900
2500,3.773700
3000,3.741100
3500,3.709000
4000,3.688100
4500,3.667300
5000,3.641000


In [ ]:
trainer.save_model()

## Test the model
To test the model I'm going to use the [pipeline](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) object from the [transformers library](https://huggingface.co/transformers/index.html). With this API it is simple to generate articles texts like this.

In [22]:
ix_robo_author = pipeline(
    'text-generation',
    model=output_model_path, 
    tokenizer='dbmdz/german-gpt2',
    config={'max_length':800}
)

In [23]:
ix_robo_author('Das neue iPhone 13')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Das neue iPhone 13 präsentiert sich von Haus aus inklusive Design mit eigenem iPhone und bietet ein voll geöffneteres Browser-Fenster. Hinzu gesellen sich der iWorkspace-Server und der App Store. Das neue iPhone 13 bietet viele neue Funktionen'

In [24]:
ix_robo_author('Die quelloffene Maschine-Learning-Bibliothek PyThorch ist')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Die quelloffene Maschine-Learning-Bibliothek PyThorch ist fertig, und auch die Open-Source-Firma Qt (www.qt.org) wird Pythorch nutzen. Mit dem Open-Source-Werkzeug können Entwickler'

In [44]:
ix_robo_author('Mit Hilfe von Apache Spark')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Mit Hilfe von Apache Spark erhält man von einer Web-Frontend-Engine, die aus dem eigenen Webportal heraus Web Services ausführen kann (siehe Abbildungˇ3): –ˇMapService: Apache Spark verbindet eine Reihe von Daten'

In [46]:
ix_robo_author('Apples neue Augmented Reallity Brille')[0]['generated_text']


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Apples neue Augmented Reallity Brille und Brillen mit GPUs: HoloLensˇ4 Mobile Computing mit  dem HoloLensˇ4 HoloLens auf der Cebit: Im Trend:'

In [48]:
ix_robo_author('Professionelle Softwareentwicklung')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Professionelle Softwareentwicklung mit Eclipse, Teil 1 - Softwareentwicklung für Java \n MARKT + TRENDS  : Objektorientierung   Objektorientierung  Metaex: objektorientiertes Entwicklungssystem Verteilte Software mit XML und DB'

In [49]:
ix_robo_author('Der beste Editor für die Entwicklung von Python')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Der beste Editor für die Entwicklung von Python bietet Pydev. Das macht Python auch für Nichtprogrammierer interessant: Pydev kann über Pydev einfach Pydev-Code ausführen und sogar Kommandozeilentools nutzen. Die meisten der beschriebenen Features'

In [53]:
ix_robo_author('Mit Hilfe von Cloud Computing')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Mit Hilfe von Cloud Computing lassen sich für kleine und mittlere Entwicklungsprojekte (KMU) mit geringem Aufwand eigene Anwendungen schaffen. Das Unternehmen mit dem Fokus auf eine serviceorientierte Architektur ist bereits seit 2016 aktiv; es bietet mit dem Werkzeug SOA Suite den Zugriff'

In [71]:
ix_robo_author('Linus Torvalds, Erfinder von Linux,')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Linus Torvalds, Erfinder von Linux, entwickelte ein Kryptohardware-System, mit dem sich Kryptographie durch eine AES-Verschlüsselung sichern lässt. Das System benötigt dazu nur 1\xa0MByte RAM und ist für die'